In [4]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="sk-or-v1-**",
)

completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
  },
  extra_body={},
  model="qwen/qwen-2.5-72b-instruct:free",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ]
)
print(completion.choices[0].message.content)

The meaning of life is a question that has puzzled humans for centuries, and the answer may vary depending on who you ask. Some people believe that the meaning of life is to seek happiness and fulfillment, while others may believe it is to serve a higher power or purpose.

From a philosophical standpoint, the meaning of life is a subjective concept and can be defined by each individual based on their values, beliefs, and experiences. For example, some may find meaning in personal growth, helping others, or creating something that will have a lasting impact on the world.

It's important to note that there is no single, definitive answer to this question, and individuals may have different perspectives on what gives their life meaning. Ultimately, it is up to each person to determine their own purpose and find meaning in their own way.


In [72]:
import re
import json
import logging

# Настройка логирования в консоль
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s [%(levelname)s] %(message)s")

# Для отображения логов прямо в ноутбуке, можно использовать:
logger = logging.getLogger()


In [73]:
def fake_model(prompt, max_tokens, temperature, stop):
    # Симулируем ответ модели, как если бы она вернула корректный JSON с разметкой
    # В данном случае вместо обрамления в ```json мы возвращаем просто JSON
    response_text = """{
  "nodes": [
    {"id": "s1", "name": "Начало", "type": "start"},
    {"id": "s2", "name": "Проверка наличия лекарства", "type": "task"},
    {"id": "s3", "name": "Ветвление", "type": "gateway"},
    {"id": "s4", "name": "Заказать лекарство", "type": "task"},
    {"id": "s5", "name": "Подтвердить заказ", "type": "task"},
    {"id": "s6", "name": "Получить лекарство", "type": "task"},
    {"id": "s7", "name": "Завершение", "type": "end"}
  ],
  "flows": [
    {"source": "s1", "target": "s2"},
    {"source": "s2", "target": "s3"},
    {"source": "s3", "target": "s4"},
    {"source": "s3", "target": "s5"},
    {"source": "s4", "target": "s6"},
    {"source": "s5", "target": "s6"},
    {"source": "s6", "target": "s7"}
  ]
}"""
    # Возвращаем результат в том же формате, что ожидает наш агент
    return {"choices": [{"text": response_text}]}


In [74]:
# Фейковая модель для тестирования: возвращает заранее заданный JSON-ответ
def fake_model(prompt, max_tokens, temperature, stop):
    response_text = """{
  "nodes": [
    {"id": "s1", "name": "Начало", "type": "start"},
    {"id": "s2", "name": "Проверка наличия лекарства", "type": "task"},
    {"id": "s3", "name": "Ветвление", "type": "gateway"},
    {"id": "s4", "name": "Заказать лекарство", "type": "task"},
    {"id": "s5", "name": "Подтвердить заказ", "type": "task"},
    {"id": "s6", "name": "Получить лекарство", "type": "task"},
    {"id": "s7", "name": "Завершение", "type": "end"}
  ],
  "flows": [
    {"source": "s1", "target": "s2"},
    {"source": "s2", "target": "s3"},
    {"source": "s3", "target": "s4"},
    {"source": "s3", "target": "s5"},
    {"source": "s4", "target": "s6"},
    {"source": "s5", "target": "s6"},
    {"source": "s6", "target": "s7"}
  ]
}"""
    return {"choices": [{"text": response_text}]}

# Определение агента для генерации цепочки событий
class EventChainAgent:
    def __init__(self, model):
        self.model = model

    def generate_chain(self, process_description):
        prompt = f"""
ВАЖНО:
- Каждый шаг должен быть объектом с полями: id (строка), name (строка), type (start/end/task/gateway)
- Каждая связь — объект с полями source и target (id шагов)
- Ответ должен быть ТОЛЬКО JSON
- Никаких пояснений, только JSON-объект
- Пример:
{{
  "nodes": [
    {{"id": "s1", "name": "Начало", "type": "start"}},
    {{"id": "s3", "name": "Ветвление", "type": "gateway"}}
  ],
  "flows": [{{"source": "s1", "target": "s3"}}]
}}

Описание процесса: {process_description}

ВАЖНО: Ответ должен быть только JSON без лишних символов!
```json
"""
        logging.debug("Формирование промпта для генерации цепочки:\n%s", prompt)
        response = self.model(
            prompt,
            max_tokens=1024,
            temperature=0.3,
            stop=["<|im_end|>"]
        )
        content = response["choices"][0]["text"].strip()
        logging.debug("Сырой ответ модели:\n%s", content)
        
        json_str = self._extract_json(content)
        logging.debug("Выделенный JSON-строковый блок:\n%s", json_str)
        
        data = self._validate_json(json_str)
        logging.debug("Валидированный event_chain:\n%s", data)
        return data

    def _extract_json(self, text: str) -> str:
        # Пытаемся извлечь JSON между блоками ```json и ```
        match = re.search(r"```json\s*(\{.*\})\s*```", text, re.DOTALL)
        if match:
            logging.debug("JSON успешно выделен из блока ```json```.")
            return match.group(1).strip()
        else:
            logging.debug("Блок ```json``` не найден. Пытаемся выделить JSON по позициям фигурных скобок.")
            first = text.find("{")
            last = text.rfind("}")
            if first != -1 and last != -1 and last > first:
                result = text[first:last+1].strip()
                logging.debug("JSON выделен по позициям фигурных скобок:\n%s", result)
                return result
        logging.error("Не удалось извлечь корректный JSON из текста:\n%s", text)
        raise ValueError("Не удалось извлечь корректный JSON из ответа модели.")

    def _validate_json(self, json_str: str) -> dict:
        try:
            data = json.loads(json_str)
            logging.debug("Преобразование JSON-строки в dict прошло успешно:\n%s", data)
            if "nodes" not in data or "flows" not in data:
                logging.error("Проверка структуры: отсутствуют ключи 'nodes' или 'flows' в:\n%s", data)
                raise ValueError("Отсутствуют ключи 'nodes' или 'flows'.")
            for idx, step in enumerate(data["nodes"]):
                if "id" not in step or "name" not in step or "type" not in step:
                    logging.error("Проверка узла #%d: отсутствуют обязательные поля в:\n%s", idx, step)
                    raise ValueError("Отсутствуют обязательные поля в одном из узлов.")
            for idx, flow in enumerate(data["flows"]):
                if "source" not in flow or "target" not in flow:
                    logging.error("Проверка потока #%d: отсутствуют обязательные поля в:\n%s", idx, flow)
                    raise ValueError("Отсутствуют обязательные поля в одном из потоков.")
            return data
        except Exception as e:
            logging.error("Ошибка при валидации JSON: %s. Исходная строка:\n%s", str(e), json_str)
            raise ValueError(f"Ошибка валидации: {str(e)}")

# Тест генерации цепочки событий
process_description = "аптека"
agent = EventChainAgent(fake_model)
event_chain = agent.generate_chain(process_description)
print("Сгенерированная цепочка событий:")
print(json.dumps(event_chain, indent=2, ensure_ascii=False))


2025-04-12 10:06:06,351 [DEBUG] Формирование промпта для генерации цепочки:

ВАЖНО:
- Каждый шаг должен быть объектом с полями: id (строка), name (строка), type (start/end/task/gateway)
- Каждая связь — объект с полями source и target (id шагов)
- Ответ должен быть ТОЛЬКО JSON
- Никаких пояснений, только JSON-объект
- Пример:
{
  "nodes": [
    {"id": "s1", "name": "Начало", "type": "start"},
    {"id": "s3", "name": "Ветвление", "type": "gateway"}
  ],
  "flows": [{"source": "s1", "target": "s3"}]
}

Описание процесса: аптека

ВАЖНО: Ответ должен быть только JSON без лишних символов!
```json

2025-04-12 10:06:06,352 [DEBUG] Сырой ответ модели:
{
  "nodes": [
    {"id": "s1", "name": "Начало", "type": "start"},
    {"id": "s2", "name": "Проверка наличия лекарства", "type": "task"},
    {"id": "s3", "name": "Ветвление", "type": "gateway"},
    {"id": "s4", "name": "Заказать лекарство", "type": "task"},
    {"id": "s5", "name": "Подтвердить заказ", "type": "task"},
    {"id": "s6", "name

Сгенерированная цепочка событий:
{
  "nodes": [
    {
      "id": "s1",
      "name": "Начало",
      "type": "start"
    },
    {
      "id": "s2",
      "name": "Проверка наличия лекарства",
      "type": "task"
    },
    {
      "id": "s3",
      "name": "Ветвление",
      "type": "gateway"
    },
    {
      "id": "s4",
      "name": "Заказать лекарство",
      "type": "task"
    },
    {
      "id": "s5",
      "name": "Подтвердить заказ",
      "type": "task"
    },
    {
      "id": "s6",
      "name": "Получить лекарство",
      "type": "task"
    },
    {
      "id": "s7",
      "name": "Завершение",
      "type": "end"
    }
  ],
  "flows": [
    {
      "source": "s1",
      "target": "s2"
    },
    {
      "source": "s2",
      "target": "s3"
    },
    {
      "source": "s3",
      "target": "s4"
    },
    {
      "source": "s3",
      "target": "s5"
    },
    {
      "source": "s4",
      "target": "s6"
    },
    {
      "source": "s5",
      "target": "s6"
    },
  

In [75]:
import os
import logging
import shutil
from bpmn_python.bpmn_diagram_rep import BpmnDiagramGraph

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

class BPMNAgent:
    def __init__(self, model=None):
        self.model = model

    def generate_bpmn(self, bpmn_data) -> str:
        logging.info("Начало создания BPMN-диаграммы.")
        bpmn_graph = BpmnDiagramGraph()
        
        # Инициализация диаграммы
        bpmn_graph.create_new_diagram_graph(diagram_name="Main Diagram")
        
        # Создание процесса
        process_id = bpmn_graph.add_process_to_diagram(process_name="Main Process")
        
        node_id_mapping = {}
        
        # Добавление узлов
        for node in bpmn_data.get("nodes", []):
            node_id = node["id"]
            node_type = node["type"]
            node_name = node.get("name", "")
            
            try:
                if node_type == "start":
                    new_id, _ = bpmn_graph.add_start_event_to_diagram(
                        process_id,
                        node_name,
                        start_event_definition="message",
                        parallel_multiple=False,
                        is_interrupting=True
                    )
                elif node_type == "end":
                    new_id, _ = bpmn_graph.add_end_event_to_diagram(
                        process_id,
                        node_name,
                        end_event_definition="terminate"
                    )
                elif node_type == "gateway":
                    new_id, _ = bpmn_graph.add_parallel_gateway_to_diagram(process_id, node_name)
                elif node_type in ["task", "payment", "optional"]:
                    new_id, _ = bpmn_graph.add_task_to_diagram(process_id, node_name)
                else:
                    logging.warning("Неизвестный тип узла: %s. Добавляется как 'task'.", node_type)
                    new_id, _ = bpmn_graph.add_task_to_diagram(process_id, node_name)
                
                node_id_mapping[node_id] = new_id
                logging.info("Добавлен узел '%s' с новым id '%s'.", node_name, new_id)
            except Exception as e:
                logging.error("Ошибка при добавлении узла '%s': %s", node_name, e)
        
        # Добавление связей
        for flow in bpmn_data.get("flows", []):
            source = flow["source"]
            target = flow["target"]
            source_id = node_id_mapping.get(source)
            target_id = node_id_mapping.get(target)
            
            if source_id is None or target_id is None:
                logging.warning("Пропущена связь: source=%s, target=%s", source, target)
                continue
            
            try:
                bpmn_graph.add_sequence_flow_to_diagram(
                    process_id,
                    source_id,
                    target_id,
                    sequence_flow_name="Поток"
                )
                logging.info("Добавлена связь от '%s' к '%s'.", source_id, target_id)
            except Exception as e:
                logging.error("Ошибка при добавлении связи: %s", e)
        
        # Подготовка директории для экспорта
        desired_directory = "exported_diagrams/"
        os.makedirs(desired_directory, exist_ok=True)
        
        output_filename = "diagram"  # Имя файла без расширения
        desired_path = os.path.join(desired_directory, f"{output_filename}.bpmn")
        
        # Логирование путей
        logging.info("Директория для экспорта: %s", desired_directory)
        logging.info("Имя файла: %s", output_filename)
        logging.info("Ожидаемый путь: %s", desired_path)
        # Экспорт диаграммы
        try:
            bpmn_graph.export_xml_file(desired_directory, output_filename)
            logging.info("Файл экспортирован. Проверка содержимого директории...")
            
            # Вывод содержимого директории для диагностики
            try:
                files_in_dir = os.listdir(desired_directory)
                logging.info("Файлы в директории '%s': %s", desired_directory, files_in_dir)
            except Exception as e:
                logging.error("Ошибка при чтении директории: %s", e)
            
            # Проверка наличия файла (библиотека может добавить расширение)
            if not os.path.exists(desired_path):
                possible_paths = [
                    os.path.join(desired_directory, f".{output_filename}.bpmn"),
                    os.path.join(desired_directory, output_filename),  # без расширения
                    os.path.join(desired_directory, f"{output_filename}.bpmn.bpmn"),  # двойное расширение
                    os.path.join(desired_directory, f"{output_filename}.xml")  # альтернативное расширение
                ]
                
                found_path = None
                for path in possible_paths:
                    if os.path.exists(path):
                        found_path = path
                        break
                
                if found_path:
                    # Переименовываем в корректный путь
                    os.rename(found_path, desired_path)
                    logging.info("Файл переименован из %s в %s", found_path, desired_path)
                else:
                    raise FileNotFoundError(f"Не найден файл ни по одному из путей: {possible_paths}")
            
            # Чтение содержимого файла
            with open(desired_path, "r", encoding="utf-8") as f:
                bpmn_xml = f.read()
            logging.info("BPMN-диаграмма успешно получена из %s", desired_path)
            return bpmn_xml
        except Exception as e:
            logging.error("Ошибка при экспорте: %s", e)
            raise e
bpmn_agent = BPMNAgent()
bpmn_xml = bpmn_agent.generate_bpmn(event_chain)
print("Сгенерированный BPMN XML:")
print(bpmn_xml)


2025-04-12 10:06:06,376 [INFO] Начало создания BPMN-диаграммы.
2025-04-12 10:06:06,378 [INFO] Добавлен узел 'Начало' с новым id 'iddb4ecf63-0e87-46c3-b3d4-6048847ee6bd'.
2025-04-12 10:06:06,379 [INFO] Добавлен узел 'Проверка наличия лекарства' с новым id 'id125aa447-26f1-4c54-9693-ffdce71ba9df'.
2025-04-12 10:06:06,380 [INFO] Добавлен узел 'Ветвление' с новым id 'id27f11407-3b63-434a-826b-0bb4a1af83d4'.
2025-04-12 10:06:06,381 [INFO] Добавлен узел 'Заказать лекарство' с новым id 'id65071884-c26c-47f4-a691-f6f189e61ae6'.
2025-04-12 10:06:06,382 [INFO] Добавлен узел 'Подтвердить заказ' с новым id 'id09322096-9f48-4664-a8e9-455342922965'.
2025-04-12 10:06:06,383 [INFO] Добавлен узел 'Получить лекарство' с новым id 'id6b1128e2-0578-4480-b3de-a2148abaefe9'.
2025-04-12 10:06:06,384 [INFO] Добавлен узел 'Завершение' с новым id 'id002582d0-1eac-4f8a-b17e-05e2f0c4b898'.
2025-04-12 10:06:06,385 [INFO] Добавлена связь от 'iddb4ecf63-0e87-46c3-b3d4-6048847ee6bd' к 'id125aa447-26f1-4c54-9693-ffdce7

2025-04-12 10:06:06,387 [INFO] Добавлена связь от 'id27f11407-3b63-434a-826b-0bb4a1af83d4' к 'id65071884-c26c-47f4-a691-f6f189e61ae6'.
2025-04-12 10:06:06,388 [INFO] Добавлена связь от 'id27f11407-3b63-434a-826b-0bb4a1af83d4' к 'id09322096-9f48-4664-a8e9-455342922965'.
2025-04-12 10:06:06,390 [INFO] Добавлена связь от 'id65071884-c26c-47f4-a691-f6f189e61ae6' к 'id6b1128e2-0578-4480-b3de-a2148abaefe9'.
2025-04-12 10:06:06,391 [INFO] Добавлена связь от 'id09322096-9f48-4664-a8e9-455342922965' к 'id6b1128e2-0578-4480-b3de-a2148abaefe9'.
2025-04-12 10:06:06,391 [INFO] Добавлена связь от 'id6b1128e2-0578-4480-b3de-a2148abaefe9' к 'id002582d0-1eac-4f8a-b17e-05e2f0c4b898'.
2025-04-12 10:06:06,393 [INFO] Директория для экспорта: exported_diagrams/
2025-04-12 10:06:06,393 [INFO] Имя файла: diagram
2025-04-12 10:06:06,394 [INFO] Ожидаемый путь: exported_diagrams/diagram.bpmn
2025-04-12 10:06:06,396 [INFO] Файл экспортирован. Проверка содержимого директории...
2025-04-12 10:06:06,397 [INFO] Файлы

Сгенерированный BPMN XML:
<?xml version='1.0' encoding='utf-8'?>
<definitions xmlns="http://www.omg.org/spec/BPMN/20100524/MODEL" xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI" xmlns:omgdc="http://www.omg.org/spec/DD/20100524/DC" xmlns:omgdi="http://www.omg.org/spec/DD/20100524/DI" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" targetNamespace="http://www.signavio.com/bpmn20" typeLanguage="http://www.w3.org/2001/XMLSchema" expressionLanguage="http://www.w3.org/1999/XPath" xmlns:xsd="http://www.w3.org/2001/XMLSchema">
  <bpmndi:BPMNDiagram id="idd88ea040-9933-45df-82e3-2b4befb17e96" name="Main Diagram">
    <bpmndi:BPMNPlane id="id162a60f7-a093-4489-a611-688cc6a5053c" bpmnElement="id1434c402-64df-4f3e-a8dd-b0da3533e4f8">
      <bpmndi:BPMNShape id="iddb4ecf63-0e87-46c3-b3d4-6048847ee6bd_gui" bpmnElement="iddb4ecf63-0e87-46c3-b3d4-6048847ee6bd">
        <omgdc:Bounds width="100" height="100" x="100" y="100" />
      </bpmndi:BPMNShape>
    <bpmndi:BPMNShape id="id125aa447

In [13]:
# event_chain_agent.py
import re
import json
import logging
from typing import Any, Dict

logger = logging.getLogger(__name__)

class JSONParseError(Exception):
    pass

class EventChainAgent:

    def __init__(self, llm_callable: Any):
            """
            llm_callable(prompt: str, **kwargs) -> {"choices": [{"text": str}, ...]}
            """
            self.llm = llm_callable

    def generate_chain(self, process_description):
        prompt = f"""
**Вы эксперт в BPMN 2.0. Создайте максимально информативную и логичную диаграмму, используя:**
- Задачи (Tasks)
- События: start, end, intermediate
- Шлюзы: exclusive (условия), parallel (параллельные потоки)
- Последовательности потоков (Sequence Flow)

**Формат ответа:**
{{
  "nodes": [
    {{"id": "str", "name": "str", "type": "start/end/task/gateway/intermediate"}},
    {{"id": "g1", "name": "Пример шлюза", "type": "gateway", "gateway_type": "exclusive/parallel"}}
  ],
  "flows": [
    {{"source": "source_id", "target": "target_id"}}
  ]
}}

**Правила:**
1. Используйте ДВОЙНЫЕ КАВЫЧКИ для всех ключей и значений
2. Все элементы должны быть связаны корректными потоками
3. Для шлюзов (gateway) обязательно поле gateway_type
4. Пример использования intermediate события:
   {{"id": "i1", "name": "Уведомление", "type": "intermediate"}}

Описание процесса: {process_description}
ВАЖНО: Только JSON без пояснений! Проверьте валидность перед отправкой.
```json
"""
        logging.debug("Формирование промпта для генерации цепочки: %s", prompt)
        
        # Исправленный вызов API OpenAI
        response = self.llm(
            prompt=prompt,
            max_tokens=32000,
            temperature=0.8
        )
        raw = response["choices"][0]["text"]
        logger.debug("Сырой ответ LLM:\n%s", raw)
        
        json_str = self._extract_json(raw)
        logging.debug("Выделенный JSON-строковый блок: %s", json_str)
        
        data = self._validate_json(json_str)
        logging.debug("Валидированный event_chain: %s", data)
        return data
    def _extract_json(self, text: str) -> str:
        # Извлекаем JSON из блока кода
        match = re.search(r"```json\s*(.*?)\s*```", text, re.DOTALL)
        if match:
            return match.group(1).strip()
        
        # Или находим первый валидный JSON
        first = text.find("{")
        last = text.rfind("}")
        if first != -1 and last != -1 and last > first:
            return text[first:last+1].strip()
        
        raise ValueError("Не найден корректный JSON")

    def _validate_json(self, json_str: str) -> dict:
        try:
            data = json.loads(json_str)
        except json.JSONDecodeError as e:
            # Логируем позицию ошибки
            error_pos = getattr(e, 'pos', None)
            logging.error(f"Ошибка декодирования JSON на позиции {error_pos}: {e}")
            raise ValueError(f"Некорректный JSON: {str(e)}")
        
        # Проверка структуры
        if not isinstance(data, dict):
            raise ValueError("Ожидался объект JSON, но получен другой тип данных")
        
        required_keys = ["nodes", "flows"]
        if not all(k in data for k in required_keys):
            raise ValueError(f"Отсутствуют обязательные ключи: {required_keys}")
        
        # Проверка узлов
        for node in data["nodes"]:
            if not all(k in node for k in ["id", "name", "type"]):
                raise ValueError(f"Неполный узел: {node}")
            if node["type"] == "gateway" and "gateway_type" not in node:
                raise ValueError(f"Шлюз без типа: {node['id']}")
        
        # Проверка потоков
        for flow in data["flows"]:
            if not all(k in flow for k in ["source", "target"]):
                raise ValueError(f"Некорректный поток: {flow}")
        
        return data


In [20]:
from llama_cpp import Llama # Импортируем константы

MODEL_PATH = r"model\Qwen2.5-14B-Instruct-GGUF\Qwen2.5-14B-Instruct-Q4_K_M.gguf"

model = Llama(
    model_path=MODEL_PATH,
    n_gpu_layers=-1,            # Все слои на GPU [[1]]
    split_mode=1,  # Используем константу вместо строки [[2]]
    main_gpu=0,                 # Основной GPU (если несколько)
    max_tokens= 32000,
    n_ctx=32768,                # Устанавливаем контекст из модели [[3]]
    n_batch=1024,               # Оптимальный размер батча для GPU
    use_mlock=True,             # Блокировка памяти для стабильности
    offload_kqv=True,           # Выгружаем K/Q/V на GPU [[4]]
    flash_attn=True,            # Используем Flash Attention [[5]]
    verbose=False               # Отключаем лишние логи
)

In [23]:
# Убедитесь, что модель поддерживает длинные выводы
print(f"Максимальная длина контекста: {model.n_ctx()}")

Максимальная длина контекста: 32768


In [26]:
import GPUtil
print(f"Свободная видеопамять: {GPUtil.getGPUs()[0].memoryFree} MB")

Свободная видеопамять: 5378.0 MB


In [27]:
event_agent = EventChainAgent(model)
chain = event_agent.generate_chain("пиццерия")

In [34]:
import GPUtil
gpus = GPUtil.getGPUs()
discrete_gpu = max(gpus, key=lambda x: x.memoryFree)
print(discrete_gpu.id)

0


In [28]:
print(chain)

{'nodes': [{'id': 'start', 'name': 'Начало заказа', 'type': 'start'}, {'id': 'task1', 'name': 'Прием заказа', 'type': 'task'}, {'id': 'g1', 'name': 'Выбор топпинга', 'type': 'gateway', 'gateway_type': 'exclusive'}, {'id': 'task2', 'name': 'Подготовка пиццы', 'type': 'task'}, {'id': 'task3', 'name': 'Выпечка пиццы', 'type': 'task'}, {'id': 'task4', 'name': 'Вынос пиццы', 'type': 'task'}, {'id': 'end', 'name': 'Завершение заказа', 'type': 'end'}, {'id': 'i1', 'name': 'Уведомление', 'type': 'intermediate'}], 'flows': [{'source': 'start', 'target': 'task1'}, {'source': 'task1', 'target': 'g1'}, {'source': 'g1', 'target': 'task2'}, {'source': 'task2', 'target': 'task3'}, {'source': 'task3', 'target': 'task4'}, {'source': 'task4', 'target': 'i1'}, {'source': 'i1', 'target': 'end'}]}
